In [20]:
import sys
sys.path.append("/home/yons/Desktop/developer-luo/SWatNet/models")
sys.path.append("/home/yons/Desktop/developer-luo/SWatNet/trainer")
sys.path.append("/home/yons/Desktop/developer-luo/SWatNet")
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
import data_loader
import config
from seg_models.unet import unet
from base_models.mobilenet import mobilenet_v2
from seg_models.deeplabv3_plus import deeplabv3_plus

In [21]:
mirrored_strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(mirrored_strategy.num_replicas_in_sync))


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2


In [22]:
# BATCH_SIZE_PER_REPLICA = 8
# global_batch_size = (BATCH_SIZE_PER_REPLICA *
#                      mirrored_strategy.num_replicas_in_sync)
# global_batch_size


In [23]:
# dataset = tf.data.Dataset.from_tensors(([[[1,2,3],[3,4,5]],[[1.,2,3],[3,4,5]]], [[[1.],[3,]],[[1.],[3]]]))\
#         .repeat(1000)\
#         .batch(global_batch_size)
# dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)
# dataset

dataset = data_loader.get_eva_dset()
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)
dataset


<BatchDataset shapes: (((None, 256, 256, 4), (None, 256, 256, 4), (None, 256, 256, 4)), (None, 256, 256, 1)), types: ((tf.float32, tf.float32, tf.float32), tf.float32)>

In [24]:
class model_c(tf.keras.Model):

    def __init__(self):
        super(model_c, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=1, kernel_size=1)
        self.bn = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters=1, kernel_size=1)

    def call(self, x, training=True):
        x = self.conv1(x)
        # x = self.bn(x, training=training)
        x = self.conv2(x)
        return x

def model(inputs_shape=(256,256,4)):
    inputs = tf.keras.Input(shape=inputs_shape, name="img")
    x = tf.keras.layers.Conv2D(filters=1, kernel_size=1)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(filters=1, kernel_size=1)(x)
    model = tf.keras.Model(inputs=inputs, outputs=x, name='model')
    return model

# aa = tf.ones([1, 256, 256, 4],tf.float32)
# model = model_c()
# _ = model(aa, training=False)
# model.summary()
# model.trainable_variables

In [25]:
with mirrored_strategy.scope():
    model = model_c()
    optimizer = tf.keras.optimizers.SGD()

# with mirrored_strategy.scope():
#     model = unet(nclass=2)
#     optimizer = tf.keras.optimizers.Adam()


In [26]:
# input = tf.ones([1, 256, 256, 4],tf.float32)
# _ = model(input)
# model.summary()


In [27]:
loss_object = tf.keras.losses.BinaryCrossentropy(
  from_logits=False,
  reduction=tf.keras.losses.Reduction.NONE)

def compute_loss(labels, predictions):
    per_example_loss = loss_object(labels, predictions)
    return tf.nn.compute_average_loss(per_example_loss, global_batch_size=config.BATCH_SIZE)

def train_step(inputs):
    features, labels = inputs

    with tf.GradientTape() as tape:
        predictions = model(features[2], training=True)
        loss = compute_loss(labels, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

@tf.function
def distributed_train_step(dist_inputs):
    per_replica_losses = mirrored_strategy.run(train_step, args=(dist_inputs,))
    return mirrored_strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)


In [28]:
for epoch in range(10):
    print(epoch)
    for dist_inputs in dist_dataset:
        print(distributed_train_step(dist_inputs))


0
INFO:tensorflow:batch_all_reduce: 4 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 4 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
tf.Tensor(58348.965, shape=(), dtype=float32)
tf.Tensor(425811.47, shape=(), dtype=float32)
tf.Tensor(213600.84, shape=(), dtype=float32)
tf.Tensor(28773.314, shape=(), dtype=float32)
tf.Tensor(531838.6, shape=(), dtype=float32)
tf.Tensor(257137.75, shape=(), dtype=float32)
tf.Tensor(270952.7, shape=(), dtype=float32)
tf.Tensor(177672.25, shape=(), dtype=float32)
tf.Tensor(368999.44, shape=(), dtype=float32)
tf.Tensor(502864.75, shape=(), dtype=float32)
tf.Tensor(231898.69, shape=(), dtype=float32)
tf.Tensor(472977.0, shape=(), dtype=float32)